<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Deep Learning Para Aplicações de Inteligência Artificial com Python e C++</font>
## <font color='blue'>Projeto 2</font>
## <font color='blue'>Fine-Tuning de Um Modelo Transformer Para Classificação de Sentimento</font>

In [1]:
import sys
print(sys.executable)

/opt/homebrew/opt/python@3.12/bin/python3.12


In [2]:
!{sys.executable} -m pip install spacy


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [3]:
!{sys.executable} -m pip install transformers


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [4]:
!{sys.executable} -m pip install -q -U watermark


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [5]:
!{sys.executable} -m pip install -q -U nltk


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [6]:
!{sys.executable} -m pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [7]:
!{sys.executable} -m pip install tensorflow


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [8]:
!{sys.executable} -m pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [9]:
!{sys.executable} -m pip install keras_preprocessing


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [10]:
# Imports
import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall, AUC
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
df_train = pd.read_csv("train_data.txt", header=None, delimiter=";").rename(columns={0: "text", 1: "label"})
print(f"Train data shape: {df_train.shape}")
df_train.head()

Train data shape: (16000, 2)


,text,label
0,i am feeling completely overwhelmed i have two...,fear
1,i have the feeling she was amused and delighted,joy
2,i was able to help chai lifeline with your sup...,joy
3,i already feel like i fucked up though because...,anger
4,i still love my so and wish the best for him i...,sadness


In [12]:
df_train['label'].value_counts()

label
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [13]:
df_test = pd.read_csv("test_data.txt", header=None, delimiter=";").rename(columns={0: "text", 1: "label"})
print(f"Test data shape: {df_test.shape}")
df_test.head()

Test data shape: (2000, 2)


,text,label
0,i feel like my only role now would be to tear ...,sadness
1,i feel just bcoz a fight we get mad to each ot...,anger
2,i feel like reds and purples are just so rich ...,joy
3,im not sure the feeling of loss will ever go a...,sadness
4,i feel like ive gotten to know many of you thr...,joy


In [14]:
df_train['label'].value_counts()

label
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [15]:
!{sys.executable} -m spacy download en_core_web_md -q


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [16]:
project_nlp = spacy.load("en_core_web_md")

def text_process(text):
    # Removing stopwords, extracting the stem of a word, etc
    doc = project_nlp(text)
    
    # Lemmatization
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]
    
    return " ".join(tokens)

In [17]:
df_train["processed_text"] = df_train["text"].apply(text_process)
df_train.head()

,text,label,processed_text
0,i am feeling completely overwhelmed i have two...,fear,feel completely overwhelmed strategy help feel...
1,i have the feeling she was amused and delighted,joy,feeling amuse delight
2,i was able to help chai lifeline with your sup...,joy,able help chai lifeline support encouragement ...
3,i already feel like i fucked up though because...,anger,feel like fuck not usually eat morning
4,i still love my so and wish the best for him i...,sadness,love wish good long tolerate effect bm life fa...


In [18]:
df_test["processed_text"] = df_test["text"].apply(text_process)
df_test.head()

,text,label,processed_text
0,i feel like my only role now would be to tear ...,sadness,feel like role tear sail pessimism discontent
1,i feel just bcoz a fight we get mad to each ot...,anger,feel bcoz fight mad n u wanna publicity n let ...
2,i feel like reds and purples are just so rich ...,joy,feel like red purple rich kind perfect
3,im not sure the feeling of loss will ever go a...,sadness,m sure feeling loss away dull sweet feeling no...
4,i feel like ive gotten to know many of you thr...,joy,feel like ve get know comment email m apprecia...


# 1st Approach: Using a Fully Connected Neural Network FCNN

In [19]:
# Term Frequency-Inverse Document Frequency (TF-IDF) Vectorizer
tdidf = TfidfVectorizer(max_df=0.95,
        min_df=2, 
        stop_words="english",
    )

# Applying the vetorizer
df_train_tfidf = tdidf.fit_transform(df_train["processed_text"])
# Every single transformation applied on the training data should be applied on the test data
df_test_tfidf = tdidf.transform(df_test["processed_text"])
# And, on new data, after the deploy

In [20]:
df_train_tfidf.shape

(16000, 5587)

In [22]:
type(df_train_tfidf)

scipy.sparse._csr.csr_matrix

In [21]:
# Converting the input data to array
X_train_array = df_train_tfidf.toarray()
X_test_array = df_test_tfidf.toarray()

In [26]:
# Converting the features to a numeric representation

# Creating the label encoder
label_encoder = LabelEncoder()

# Transforming the target feature 
y_train_le = label_encoder.fit_transform(df_train["label"])
y_test_le = label_encoder.transform(df_test["label"])

# Dealing with the class imbalance

class_weights = compute_class_weight("balanced", 
    classes=np.unique(y_train_le), 
    y=y_train_le,
)

In [27]:
type(class_weights)

numpy.ndarray